In [95]:
# read data
# build T-learner 
# impute values using T-learner as ground truth 
# run bootstrap ci

### End goal: compare standard CI vs bca CI

In [159]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from scipy.stats import norm, sem
from scipy.interpolate import UnivariateSpline
from sklearn.calibration import calibration_curve
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats.stats import pearsonr
from numpy.random import default_rng
import random
pd.set_option('display.max_columns', 100)

In [182]:
df

,sex,yob,g2000,g2002,g2004,p2000,p2002,p2004,treatment,cluster,voted,hh_id,hh_size,numberofnames,p2004_mean,g2004_mean
0,male,1941,yes,yes,yes,no,yes,No,Civic Duty,1,No,1.0,2,21,0.095238,0.857143
1,female,1947,yes,yes,yes,no,yes,No,Civic Duty,1,No,1.0,2,21,0.095238,0.857143
2,male,1951,yes,yes,yes,no,yes,No,Hawthorne,1,Yes,2.0,3,21,0.047619,0.857143
3,female,1950,yes,yes,yes,no,yes,No,Hawthorne,1,Yes,2.0,3,21,0.047619,0.857143
4,female,1982,yes,yes,yes,no,yes,No,Hawthorne,1,Yes,2.0,3,21,0.047619,0.857143
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
344079,male,1943,yes,yes,yes,no,yes,Yes,Control,10000,Yes,179998.0,2,21,0.476191,0.952381
344080,male,1955,yes,yes,yes,yes,no,Yes,Control,10000,No,179999.0,2,21,0.238095,0.857143
344081,female,1958,yes,no,yes,no,no,No,Control,10000,No,179999.0,2,21,0.238095,0.857143
344082,male,1937,yes,yes,yes,yes,yes,Yes,Control,10000,Yes,180000.0,2,21,0.333333,0.857143


In [199]:
df = pd.read_csv('GerberGreenLarimer_APSR_2008_social_pressure.csv')
df['treatment'] = np.where(df.treatment == ' Control',0,1)
df['voted'] = np.where(df.voted == 'Yes', 1, 0)
df['sex'] = np.where(df.sex == 'male',1, 0)
df['g2000'] = np.where(df.g2000 == 'yes', 1, 0)
df['g2002'] = np.where(df.g2002 == 'yes', 1, 0)
df['g2004'] = np.where(df.g2004 == 'yes', 1, 0)
df['p2000'] = np.where(df.p2000 == 'yes', 1, 0)
df['p2002'] = np.where(df.p2002 == 'yes', 1, 0)
df['p2004'] = np.where(df.p2004 == 'Yes', 1, 0)

In [200]:
cts_variables_names = ["yob","treatment","cluster","hh_id","hh_size","numberofnames","p2004_mean","g2004_mean"]
binary_variables_names = ["sex","g2000", "g2002", "p2000", "p2002", "p2004"]
# for column in binary_variables_names:
#     if column == 'sex':
#         df[column] = np.where(df[column] == ' male',1,0)
#     else:
#         df[column] = df[column].str.lower()
#         df[column] = np.where(df[column] == ' yes',1,0)
scaled_cts_covariates = StandardScaler().fit_transform(df[cts_variables_names])
binary_covariates = df[binary_variables_names]
d = pd.DataFrame(np.concatenate((scaled_cts_covariates, binary_covariates), axis=1), 
                        columns=cts_variables_names+binary_variables_names, index=df.index)
d["W"] = df["treatment"]
d["Y"] = df["voted"]


In [201]:
d[d['W'] == 1]

,yob,treatment,cluster,hh_id,hh_size,numberofnames,p2004_mean,g2004_mean,sex,g2000,g2002,p2000,p2002,p2004,W,Y
0,-1.052916,1.118595,-1.728468,-1.728632,-0.232742,0.138101,-1.176907,-0.914267,1.0,1.0,1.0,0.0,1.0,0.0,1,0
1,-0.637664,1.118595,-1.728468,-1.728632,-0.232742,0.138101,-1.176907,-0.914267,0.0,1.0,1.0,0.0,1.0,0.0,1,0
2,-0.360830,1.118595,-1.728468,-1.728612,1.032029,0.138101,-1.443168,-0.914267,1.0,1.0,1.0,0.0,1.0,0.0,1,1
3,-0.430038,1.118595,-1.728468,-1.728612,1.032029,0.138101,-1.443168,-0.914267,0.0,1.0,1.0,0.0,1.0,0.0,1,1
4,1.784637,1.118595,-1.728468,-1.728612,1.032029,0.138101,-1.443168,-0.914267,0.0,1.0,1.0,0.0,1.0,0.0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
344073,-1.537376,1.118595,1.736372,1.736439,-0.232742,0.138101,0.686912,0.389966,0.0,1.0,1.0,1.0,0.0,0.0,1,1
344074,-1.675793,1.118595,1.736372,1.736439,-0.232742,0.138101,0.686912,0.389966,1.0,1.0,1.0,1.0,1.0,0.0,1,1
344075,-0.637664,1.118595,1.736372,1.736459,-1.497512,0.138101,0.953172,0.389966,0.0,0.0,0.0,0.0,0.0,1.0,1,0
344076,-0.360830,1.118595,1.736372,1.736478,-0.232742,0.138101,0.953172,0.389966,0.0,1.0,1.0,0.0,1.0,0.0,1,0


In [202]:
# df = pd.read_csv('sp.csv.xz')
# cts_variables_names = ["yob", "hh_size", "totalpopulation_estimate",
#                          "percent_male", "median_age",
#                          "percent_62yearsandover",
#                          "percent_white", "percent_black",
#                          "percent_asian", "median_income",
#                          "employ_20to64", "highschool", "bach_orhigher",
#                          "percent_hispanicorlatino"]
# binary_variables_names = ["sex","g2000", "g2002", "p2000", "p2002", "p2004"]
# scaled_cts_covariates = StandardScaler().fit_transform(df[cts_variables_names])
# binary_covariates = df[binary_variables_names]
# d = pd.DataFrame(np.concatenate((scaled_cts_covariates, binary_covariates), axis=1), 
#                         columns=cts_variables_names+binary_variables_names, index=df.index)
# d["W"] = df["treat_neighbors"]
# d["Y"] = df["outcome_voted"]

In [227]:
# T learner
def T_learner(d):
    # w == 0
    d0 = d[d['W'] == 0]
    X_d0 = d0.drop(columns=['W','Y'])
    y_d0 = d0['Y']
    clf_d0 = RandomForestClassifier()
    clf_d0.fit(X_d0,y_d0)

    # w == 1
    d1 = d[d['W'] == 1]
    X_d1 = d1.drop(columns=['W','Y'])
    y_d1 = d1['Y']
    clf_d1 = RandomForestClassifier()
    clf_d1.fit(X_d1,y_d1)

    pred_y0 = clf_d0.predict_proba(d.drop(columns= ['W','Y']))[:,1]
    pred_y1 = clf_d1.predict_proba(d.drop(columns= ['W','Y']))[:,1]
    
    ate_score = (sum(pred_y1) - sum(pred_y0)) / len(pred_y0)
    return ate_score, clf_d0, clf_d1

In [228]:
full_ate, clf_d0, clf_d1 = T_learner(d)

In [229]:
d

,yob,treatment,cluster,hh_id,hh_size,numberofnames,p2004_mean,g2004_mean,sex,g2000,g2002,p2000,p2002,p2004,W,Y,new_y0,new_y1
0,-1.052916,1.118595,-1.728468,-1.728632,-0.232742,0.138101,-1.176907,-0.914267,1.0,1.0,1.0,0.0,1.0,0.0,1,0,0.0,0.0
1,-0.637664,1.118595,-1.728468,-1.728632,-0.232742,0.138101,-1.176907,-0.914267,0.0,1.0,1.0,0.0,1.0,0.0,1,0,0.0,0.0
2,-0.360830,1.118595,-1.728468,-1.728612,1.032029,0.138101,-1.443168,-0.914267,1.0,1.0,1.0,0.0,1.0,0.0,1,1,0.0,1.0
3,-0.430038,1.118595,-1.728468,-1.728612,1.032029,0.138101,-1.443168,-0.914267,0.0,1.0,1.0,0.0,1.0,0.0,1,1,0.0,1.0
4,1.784637,1.118595,-1.728468,-1.728612,1.032029,0.138101,-1.443168,-0.914267,0.0,1.0,1.0,0.0,1.0,0.0,1,1,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
344079,-0.914498,-0.893979,1.736372,1.736497,-0.232742,0.138101,0.953172,0.389966,1.0,1.0,1.0,0.0,1.0,1.0,0,1,1.0,1.0
344080,-0.083995,-0.893979,1.736372,1.736516,-0.232742,0.138101,-0.378128,-0.914267,1.0,1.0,1.0,1.0,0.0,1.0,0,0,0.0,1.0
344081,0.123630,-0.893979,1.736372,1.736516,-0.232742,0.138101,-0.378128,-0.914267,0.0,1.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0
344082,-1.329750,-0.893979,1.736372,1.736536,-0.232742,0.138101,0.154392,-0.914267,1.0,1.0,1.0,1.0,1.0,1.0,0,1,1.0,1.0


In [230]:
pred_y0 = clf_d0.predict_proba(d.drop(columns= ['W','Y','new_y0','new_y1']))[:,1]
pred_y1 = clf_d1.predict_proba(d.drop(columns= ['W','Y','new_y0','new_y1']))[:,1]

ValueError: Number of features of the model must match the input. Model n_features is 16 and input n_features is 14 

In [ ]:
ate_score = (sum(pred_y1) - sum(pred_y0)) / len(pred_y0)
ate_score

In [ ]:
sum(pred_y1)

In [235]:
new_y0 = []
new_y1 = []
for i in range(len(d)):
    if d.iloc[i].W == 0.0:
        new_y0.append(d.iloc[i].Y)
        new_y1.append(pred_y1[i])
    else:
        new_y0.append(pred_y0[i])
        new_y1.append(d.iloc[i].Y)

In [236]:
d['new_y0'] = new_y0
d['new_y1'] = new_y1

In [237]:
d

,yob,treatment,cluster,hh_id,hh_size,numberofnames,p2004_mean,g2004_mean,sex,g2000,g2002,p2000,p2002,p2004,W,Y,new_y0,new_y1
0,-1.052916,1.118595,-1.728468,-1.728632,-0.232742,0.138101,-1.176907,-0.914267,1.0,1.0,1.0,0.0,1.0,0.0,1,0,0.44,0.00
1,-0.637664,1.118595,-1.728468,-1.728632,-0.232742,0.138101,-1.176907,-0.914267,0.0,1.0,1.0,0.0,1.0,0.0,1,0,0.46,0.00
2,-0.360830,1.118595,-1.728468,-1.728612,1.032029,0.138101,-1.443168,-0.914267,1.0,1.0,1.0,0.0,1.0,0.0,1,1,0.47,1.00
3,-0.430038,1.118595,-1.728468,-1.728612,1.032029,0.138101,-1.443168,-0.914267,0.0,1.0,1.0,0.0,1.0,0.0,1,1,0.49,1.00
4,1.784637,1.118595,-1.728468,-1.728612,1.032029,0.138101,-1.443168,-0.914267,0.0,1.0,1.0,0.0,1.0,0.0,1,1,0.29,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
344079,-0.914498,-0.893979,1.736372,1.736497,-0.232742,0.138101,0.953172,0.389966,1.0,1.0,1.0,0.0,1.0,1.0,0,1,1.00,0.57
344080,-0.083995,-0.893979,1.736372,1.736516,-0.232742,0.138101,-0.378128,-0.914267,1.0,1.0,1.0,1.0,0.0,1.0,0,0,0.00,0.62
344081,0.123630,-0.893979,1.736372,1.736516,-0.232742,0.138101,-0.378128,-0.914267,0.0,1.0,0.0,0.0,0.0,0.0,0,0,0.00,0.29
344082,-1.329750,-0.893979,1.736372,1.736536,-0.232742,0.138101,0.154392,-0.914267,1.0,1.0,1.0,1.0,1.0,1.0,0,1,1.00,0.60


In [238]:
def get_bootstrap_stats(boot_estimates, full_estimates, alpha=0.05):
    
    est_stat = []
    signif_level = -norm.ppf(alpha/2)
    est_boot = np.array(boot_estimates)
    stat = {}
    stat['estimator'] = 'T-learner'
    stat['estimate'] = full_estimates
    #stat['mean_boot'] = np.mean(est_boot)
    stat['SD'] = np.std(est_boot)
    stat['CI_radius'] = signif_level * stat['SD']
    stat['lower_ci'] = stat['estimate'] - stat['CI_radius']
    stat['upper_ci'] = stat['estimate'] + stat['CI_radius']
    est_stat.append(stat)

    return pd.DataFrame(est_stat)


def get_samples(data, control_treat_prop, n_training_set = 50000, n_testing_set = 2000):
    
    # randomly select 50000 training set and 2000 testing set
    # among them, we need to make w = 1 and w = 0 according to proportion, this is the permutation we have
    idx = list(np.arange(len(data)))
    random_idx = random.sample(idx, n_training_set + n_testing_set)
    # frist 50000 are training, last 2000 are testing
    train_sample = random_idx[:n_training_set]
    test_sample = random_idx[:n_testing_set]
    train_df = data.iloc[train_sample].copy()
    test_df = data.iloc[test_sample].copy()
    
    # assgin w = 1 according to proprtion
    w_train = np.zeros(len(train_sample))
    w_train[int(len(train_sample) * control_treat_prop) :] = 1 
    w_test = np.zeros(len(test_sample))
    w_test[int(len(test_sample) * control_treat_prop) :] = 1 
    
    # choose new Y
    y_train = []
    y_test = []
    for i in range(len(w_train)):
        if w_train[i] == 0:
            y_train.append(train_df.iloc[i].new_y0)
        else:
            y_train.append(train_df.iloc[i].new_y1)
    for i in range(len(w_test)):
        if w_test[i] == 0:
            y_test.append(test_df.iloc[i].new_y0)
        else:
            y_test.append(test_df.iloc[i].new_y1)
    # add new W,Y to both dfs respectively
    train_df = train_df.drop(columns = ['W','Y','new_y0','new_y1'])
    test_df = test_df.drop(columns = ['W','Y','new_y0','new_y1'])
    train_df['W'] = w_train
    train_df['Y'] = y_train
    test_df['W'] = w_test
    test_df['Y'] = y_test
    
    return train_df, test_df


def boostrap_exp(estimator,data,control_treat_prop, n_training_set = 50000, n_testing_set = 2000 , num_exps = 10):
        bool_ates = []
        full_estimate_ate, _, _ = estimator(data)
        for _ in range(num_exps):
            train_df, test_df = get_samples(data,control_treat_prop)
            # train T-learner
            ate, clf_w0, clf_w1 = estimator(train_df)
            # calculate the ate for boot test set
            w0_result = [clf_w0.predict(np.array(test_df.iloc[i].drop(['W','Y'])).reshape(1, -1)) for i in range(len(test_df))]
            w1_result = [clf_w1.predict(np.array(test_df.iloc[i].drop(['W','Y'])).reshape(1, -1)) for i in range(len(test_df))]
            bool_ates.append((sum(w1_result) - sum(w0_result))/n_testing_set)
        return get_bootstrap_stats(bool_ates, full_estimate_ate)


In [239]:
control_treat_prop = len(d[d['W'] == 1]) / len(d)
result = boostrap_exp(T_learner, d, control_treat_prop )

ValueError: Unknown label type: 'continuous'

In [216]:
result

,estimator,estimate,SD,CI_radius,lower_ci,upper_ci
0,T-learner,0.033626,0.010053,0.019704,0.013921,0.05333
